In [1]:
# import all the libraries
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoTokenizer, AutoModel
import polars as pl

import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

import polars as pl

import xgboost as xgb
from tqdm import tqdm

In [2]:
# read the data
data = pd.read_csv("../data/pullreq_with_code.csv")

In [3]:
# drop the rows with no code
data = data[data["added_code"].astype(str) != "None"]

In [4]:
# only get the merged data, and drop any data that have n/a as gender
rejected_data = data.loc[data['merged_or_not'] == 0]
rejected_data = rejected_data.loc[rejected_data['contrib_gender'].notna()]

# drop the columns that are not needed
rejected_data = rejected_data.drop(['ownername', 'reponame', 'id', 'project_id', 'github_id', 'creator_id'], axis=1)

In [5]:
# Assign smaller class (female) 1
gender_dict = {'male': 0, 'female': 1}

rejected_data['contrib_gender'] = rejected_data['contrib_gender'].map(gender_dict)
df_majority = rejected_data[rejected_data['contrib_gender']==0]
df_minority = rejected_data[rejected_data['contrib_gender']==1]

df_majority_downsampled = df_majority.sample(len(df_minority), replace=False)

rejected_data = pd.concat([df_majority_downsampled, df_minority])

X = rejected_data['added_code']
Y = rejected_data['contrib_gender']

In [6]:
# prepare the tokenizer and model for codebert
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
codebert_model = AutoModel.from_pretrained("microsoft/codebert-base")

In [7]:
# get the embeddings for each code
X_embeddings = []
for code in tqdm(X):
    inputs = tokenizer(code, return_tensors="pt", truncation=True, padding=True)
    outputs = codebert_model(**inputs) # inputs = input_ids, attention_mask
    X_embeddings.append(outputs.last_hidden_state[:, 0, :].detach().numpy())

  0%|          | 3/1644 [00:04<39:19,  1.44s/it]


KeyboardInterrupt: 

In [8]:
# reshape the embeddings
X_embeddings = np.array(X_embeddings)
X_embeddings = X_embeddings.reshape(-1, codebert_model.config.hidden_size)

In [21]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.fc2 = nn.Linear(hidden_size, hidden_size) 
        self.fc3 = nn.Linear(hidden_size, hidden_size) 
        self.fc4 = nn.Linear(hidden_size, num_classes)  
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.fc3(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.fc4(out)
        return out

In [23]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from tqdm import tqdm

model = Net(input_size=codebert_model.config.hidden_size, hidden_size=1000, num_classes=2)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

X = np.array(X_embeddings)
y = np.array(Y)

X = X[:5000]
y = y[:5000]

kfold = StratifiedKFold(n_splits=10, shuffle=True)

smote = SMOTE()

for fold, (train_ids, test_ids) in enumerate(kfold.split(X, y)):
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    X_train, X_test = X[train_ids], X[test_ids]
    y_train, y_test = y[train_ids], y[test_ids]

    X_train, y_train = smote.fit_resample(X_train, y_train)

    for epoch in range(20):
        for i, (codes, labels) in tqdm(enumerate(zip(X_train, y_train))):
            codes = torch.tensor(codes).unsqueeze(0)
            labels = torch.tensor([labels]).long()

            outputs = model(codes)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        correct = 0
        total = 0
        y_true = []
        y_pred = []
        with torch.no_grad():
            for i, (codes, labels) in tqdm(enumerate(zip(X_test, y_test))):

                codes = torch.tensor(codes).unsqueeze(0)
                labels = torch.tensor([labels]).long()

                outputs = model(codes)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                y_true.append(labels.item())
                y_pred.append(predicted.item())

        accuracy = 100 * correct / total
        print ('Epoch [{}/{}], Train Loss: {:.4f}, Test Accuracy: {:.2f}%'.format(epoch+1, 20, loss.item(), accuracy))
        print(classification_report(y_true, y_pred, zero_division=0))

FOLD 0
--------------------------------


8162it [01:15, 107.70it/s]
500it [00:00, 3199.04it/s]


Epoch [1/20], Train Loss: 0.0000, Test Accuracy: 9.20%
              precision    recall  f1-score   support

           0       0.09      1.00      0.17        46
           1       0.00      0.00      0.00       454

    accuracy                           0.09       500
   macro avg       0.05      0.50      0.08       500
weighted avg       0.01      0.09      0.02       500



8162it [01:16, 106.82it/s]
500it [00:00, 3175.25it/s]


Epoch [2/20], Train Loss: 0.0001, Test Accuracy: 9.20%
              precision    recall  f1-score   support

           0       0.09      1.00      0.17        46
           1       0.00      0.00      0.00       454

    accuracy                           0.09       500
   macro avg       0.05      0.50      0.08       500
weighted avg       0.01      0.09      0.02       500



...

Epoch [14/20], Train Loss: 0.0001, Test Accuracy: 9.20%
              precision    recall  f1-score   support

           0       0.09      1.00      0.17        46
           1       0.00      0.00      0.00       454

    accuracy                           0.09       500
   macro avg       0.05      0.50      0.08       500
weighted avg       0.01      0.09      0.02       500



8162it [01:15, 108.81it/s]
500it [00:00, 2997.87it/s]


Epoch [15/20], Train Loss: 0.0001, Test Accuracy: 9.20%
              precision    recall  f1-score   support

           0       0.09      1.00      0.17        46
           1       0.00      0.00      0.00       454

    accuracy                           0.09       500
   macro avg       0.05      0.50      0.08       500
weighted avg       0.01      0.09      0.02       500



3905it [00:37, 103.73it/s]


KeyboardInterrupt: 